In [1]:
import json
import csv
import random
import torch
import torch.nn as nn
import numpy as np
# use this library https://github.com/facebookresearch/fastText/tree/master/python
import fastText


In [2]:
trainset = []
with open('data/intents_train.csv', 'r') as f:
    reader = csv.reader(x.replace('\0', '') for x in f)
    for line in reader:
        trainset.append(line)
trainset = np.array(trainset)

testset = []
with open('data/intents_test.csv', 'r') as f:
    reader = csv.reader(x.replace('\0', '') for x in f)
    for line in reader:
        testset.append(line)
testset = np.array(testset)

labels = list(set(trainset[:,1]))
lab2id = {}
id2lab = {}

for i in range(len(labels)):
    lab2id[labels[i]] = i
    id2lab[i] = labels[i]

In [3]:
sv_model = fastText.load_model('data/cc.sv.300.bin')
en_model = fastText.load_model('data/cc.en.300.bin')

In [51]:
def sentence_vec(sentence, model):
    result = np.zeros((1, 300))
    sentence = sentence.strip()
    for word in sentence:
        result += model.get_word_vector(word.lower())
    return result/len(sentence)
    

In [284]:
def prepare_pair(label, sentence, model):
    return (lab2id[label]),(sentence_vec(sentence, model))

def prepare_pairs(data, lang = 'en'):
    if lang == 'en':
        model = en_model
        slab = 2
    elif lang == 'sv':
        model = sv_model
        slab = 3
    else:
        raise RuntimeError('lang is not supported')
    labels = []
    vectors = []
    for sample in data:
        l, v = prepare_pair(sample[1], sample[slab], model)
        labels.append(l)
        vectors.append(v)
        
    return labels, vectors

In [294]:
class Baseline(nn.Module):
    def __init__(self, in_size = 300, out_size = 10):
        super(Baseline, self).__init__()

        self.W = nn.Linear(300, 10)
        self.out = nn.LogSoftmax(2)
        
    def forward(self, x):
        x = self.W(x)
        return self.out(x)



In [296]:
def train(model, criterion, optimizer, labels, vectors):
    model.zero_grad()
    loss = 0
    
    vectors = torch.tensor(vectors).float()
    labels = torch.tensor(labels)
    
    model_out = model.forward(vectors)
    loss += criterion(model_out[:,0], labels)
    
    loss.backward()
    optimizer.step()
    
    return loss.item()/len(labels)

In [300]:
def eval(model, labels, vectors):
    with torch.no_grad():
        vectors = torch.tensor(vectors).float()
        labels = torch.tensor(labels)
    
        model_out = model.forward(vectors)
        right = 0
        
        for i  in range(len(model_out)):
            k, v = model_out[i].topk(1)
            predicted, true = v.item(), labels[i].item()
            if predicted == true:
                right +=1

                
        loss = criterion(model_out[:,0], labels)
        return loss.item(), right/len(model_out)
    
        

In [310]:
net = Baseline()
optimizer = torch.optim.Adam(net.parameters())
criterion = torch.nn.NLLLoss()
labs, vecs = prepare_pairs(trainset, lang = 'sv')
labst, vecst = prepare_pairs(testset, lang = 'sv')


for i in range(4001):
    loss = train(net, criterion, optimizer, labs, vecs)
    if not i% 100:
        eval_loss, acc = eval(net, labst, vecst)
        print('#{:4d}, train loss: {:3f}, eval loss: {:3f}, acc = {:3f}'.format(i, loss, eval_loss, acc))
        
    

#   0, train loss: 0.007340, eval loss: 2.291807, acc = 0.038462
# 100, train loss: 0.006661, eval loss: 2.135674, acc = 0.153846
# 200, train loss: 0.006281, eval loss: 2.040863, acc = 0.179487
# 300, train loss: 0.005930, eval loss: 1.954765, acc = 0.243590
# 400, train loss: 0.005614, eval loss: 1.878127, acc = 0.294872
# 500, train loss: 0.005329, eval loss: 1.810425, acc = 0.358974
# 600, train loss: 0.005074, eval loss: 1.750744, acc = 0.384615
# 700, train loss: 0.004845, eval loss: 1.698082, acc = 0.474359
# 800, train loss: 0.004638, eval loss: 1.651480, acc = 0.500000
# 900, train loss: 0.004451, eval loss: 1.610078, acc = 0.512821
#1000, train loss: 0.004281, eval loss: 1.573125, acc = 0.525641
#1100, train loss: 0.004126, eval loss: 1.539982, acc = 0.564103
#1200, train loss: 0.003984, eval loss: 1.510109, acc = 0.576923
#1300, train loss: 0.003854, eval loss: 1.483057, acc = 0.589744
#1400, train loss: 0.003734, eval loss: 1.458445, acc = 0.602564
#1500, train loss: 0.0036

In [302]:
def eval_visual(model, labels, vectors):
    with torch.no_grad():
        vectors = torch.tensor(vectors).float()
        labels = torch.tensor(labels)
    
        model_out = model.forward(vectors)
        right = 0
        wrong = 0
        for i  in range(len(model_out)):
            k, v = model_out[i].topk(1)
            predicted, true = v.item(), labels[i].item()
            if predicted == true:
                right +=1
            else:
                print(id2lab[predicted], id2lab[true])
                wrong +=1
                
        print('{} out of {} = {}'.format(right, right+wrong, right/(right+wrong)))

In [269]:
eval_visual(net, labst, vecst)

ComparePlaces GetPlaceDetails
SearchPlace RequestRide
BookRestaurant GetWeather
GetPlaceDetails GetWeather
RequestRide GetTrafficInformation
SearchPlace GetPlaceDetails
GetTrafficInformation GetDirections
GetPlaceDetails GetWeather
GetDirections GetPlaceDetails
GetWeather GetPlaceDetails
GetDirections SearchPlace
SearchPlace ShareETA
SearchPlace GetPlaceDetails
GetPlaceDetails GetTrafficInformation
GetTrafficInformation GetWeather
GetWeather ShareCurrentLocation
GetPlaceDetails ComparePlaces
SearchPlace BookRestaurant
GetPlaceDetails GetTrafficInformation
SearchPlace RequestRide
ComparePlaces GetPlaceDetails
GetWeather GetPlaceDetails
GetDirections GetPlaceDetails
GetWeather GetPlaceDetails
GetPlaceDetails GetWeather
RequestRide GetWeather
BookRestaurant GetWeather
BookRestaurant GetTrafficInformation
50 out of 78 = 0.6410256410256411


In [4]:
trainset

array([['reservation', 'BookRestaurant',
        'Book a table for four people at Tapestry for 9pm tonight',
        'Boka ett bord för fyra personer på Väv till klockan 9 på kvällen ikväll'],
       ['reservation', 'BookRestaurant',
        'Book a table for 8 at Tavern on the Green for 8 pm',
        'Boka ett bord för 8 på Tavern on the Green för 8 pm'],
       ['weather', 'GetWeather',
        "What's the weather like at Jo's place right now?",
        'Vad är det för väder på Jo: s plats just nu?'],
       ...,
       ['places', 'ShareCurrentLocation',
        'Share my location with Robert for the next 10 min',
        'Dela min plats med Robert för nästa 10 min'],
       ['reservation', 'BookRestaurant',
        "Please book a table at Delmonico's this evening for 3 at 8pm",
        "Boka gärna bord på Delmonico ' s denna kväll för 3 vid 8pm"],
       ['places', 'GetPlaceDetails',
        "Show me the Butcher's Daughter's menu",
        "Visa mig Butcher 's Daughter' s meny"]], 